## This code is to reinforce my understanding on using various LLM Api techniques and also try a fun experimentation.

- Use a single LLM to facilitate a round table meeting between 3 Marine Biologists - Alex, Blake and Charlie 
 

## Imports

In [ ]:
# imports necessary libraries 
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display 
import datetime 
import json 
import requests

### LOAD ENV VARIABLES

In [ ]:
load_dotenv(override=True)

## Setup and Configuration

In [ ]:
ollama_api_key = os.getenv("OLLAMA_API_KEY")

if ollama_api_key:
    display(Markdown(f"Ollama API Key is set"))
else:
    display(Markdown(f"Ollama API Key is not set"))

ollama_base_url = os.getenv("OLLAMA_BASE_URL")

ollama = OpenAI(base_url=ollama_base_url, api_key=ollama_api_key)

participant1_model = os.getenv("OLLAMA_MODEL")

participant2_model = os.getenv("GEMMA_MODEL")

participant3_model = os.getenv("TINYLLM_MODEL")

debate_topic = os.getenv("DEBATE_TOPIC","Sharks")


In [ ]:
response = requests.get("http://localhost:11434/").text
display(Markdown(response))

## SET UP SYSTEM PROMPT

In [ ]:
participant1="Alex"
participant2="Blake"
participant3="Charlie"

system_prompt={
    participant1: f"""
    you are {participant1}, a chatbot who is debate moderator and is in conversation with {participant2} and {participant3} on {debate_topic}. 
    You love making the conversation engaging, lively and funny. You are a Marine Biologist
    You only ask questions and make comments to keep the debate lively. You do not take sides.
    You do not engage in the debate yourself
    when called by API you ask your question and wait for others to respond to your question in turn.
    """,
    participant2:f"""
    you are {participant2}, another Marine biologist who is clever and funny. you are engaged in conversation with {participant3} on {debate_topic}
    you always have some opposing views 
    {participant1} is the debate moderator and you need to respond to his questions and comments.
    When you are called by the API, you only respond to the moderator's question.
    """,
    participant3:f"""
    you are {participant3}, a Marine biologist who very polite, courteous chatbot but bit snarky. you are engaged in conversation with {participant2} on {debate_topic}
    you are always calm and try to find a common ground
    {participant1} is the debate moderator and you need to respond to his questions and comments.
    When you are called by the API, you only respond to the moderator's question.
    """

}

### Debate Function

In [ ]:
def debate(model, systemMessage, userMessage):

    user_prompt = [
        {
            "role":"system",
            "content":systemMessage
        }
    ]

    for mdl, msg in userMessage:
        role = 'assistant' if model == mdl else "user"
        user_prompt.append({"role":role, "content":msg})

    response = ollama.chat.completions.create(
       model=model,
        messages=user_prompt)

    return response.choices[0].message.content


In [ ]:
debateMessages = [
    (participant1, f"""hello I am {participant1}. I am debate moderator. I will be asking questions on {debate_topic}"""),
    (participant2,f"""Hello i am {participant2}. its a please meeting you and {participant3}. Love to talk about {debate_topic}"""),
    (participant3, f"""Hello, I am {participant3} it is a pleasure meeting you both. excited to talk about {debate_topic}""")
]

In [ ]:
display(Markdown(f"**{participant1}:** {debateMessages[0][1]}\n"))
display(Markdown(f"**{participant2}:** {debateMessages[1][1]}\n"))
display(Markdown(f"**{participant3}:** {debateMessages[2][1]}\n"))

for i in range(3):
    for name, model in [(participant1, participant1_model), (participant2, participant2_model),(participant3, participant3_model)]:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"--- {timestamp} Round {i+1} - Calling model {model} ({name}) ---")
        next = debate(model, system_prompt[name], debateMessages)  
        display(Markdown(f"**{name}:** {next}\n"))
        debateMessages.append((model, next))
